In [11]:
import copy

%load_ext autoreload
%autoreload 2

import os
import numpy as np
import torch

from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# From Dowhy

In [12]:
# importing required libraries
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

In [13]:
# main_dataset= pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
main_dataset= pd.read_csv("../../../CausalRFF/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
for i in range(1,26):
    col.append("x"+str(i))
main_dataset.columns = col
main_dataset['x14'] = main_dataset['x14'].replace({1: 0, 2: 1})


# Shuffle the dataset
main_dataset = main_dataset.sample(frac=1).reset_index(drop=True)
main_dataset

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,4.884405,2.035831,2.259494,6.485341,1.094769,0.596582,0.011465,-0.879606,-0.941775,...,1,1,1,0,0,0,0,0,0,0
1,0,1.517859,6.925120,1.852252,6.286602,-0.180583,0.596582,-0.360898,-0.879606,0.246052,...,1,1,1,0,0,1,0,0,0,0
2,0,2.492872,6.779199,1.979112,6.352848,0.078811,0.596582,-1.105624,-0.879606,0.808706,...,1,1,1,0,0,0,0,0,0,1
3,0,1.346275,6.927688,1.375648,5.989125,1.029921,-0.202946,-0.360898,-0.879606,-1.566947,...,1,0,1,0,0,0,1,0,0,0
4,0,2.009034,5.932199,2.586113,6.620356,0.230124,0.596582,-0.733261,-0.879606,-0.254086,...,1,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,1.411111,7.505549,1.287469,5.922879,-0.202199,1.396110,-0.733261,-0.879606,1.121292,...,1,0,1,0,0,1,0,0,0,0
743,0,3.372611,7.185988,4.406124,7.153195,-1.520784,-0.202946,0.383828,0.161703,0.433603,...,1,0,1,0,0,1,0,0,0,0
744,0,1.926075,7.483697,2.414246,6.551587,0.338204,0.996346,-0.733261,-0.879606,0.308569,...,1,1,1,0,0,0,0,0,0,1
745,0,3.252402,5.523135,2.293447,6.500256,1.246082,1.795874,-0.733261,2.244320,0.371086,...,1,1,1,0,0,0,0,0,1,0


In [14]:
del main_dataset['y_cfactual']
del main_dataset['mu0']
del main_dataset['mu1']


main_dataset= main_dataset[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7',
       'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17',
       'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'treatment', 'y_factual']] 
    
    # = main_dataset[['treatment', 'y_factual', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7',
    #    'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17',
    #    'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25']]

       
main_dataset

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x18,x19,x20,x21,x22,x23,x24,x25,treatment,y_factual
0,1.094769,0.596582,0.011465,-0.879606,-0.941775,0.632754,0,0,1,0,...,1,0,0,0,0,0,0,0,1,4.884405
1,-0.180583,0.596582,-0.360898,-0.879606,0.246052,0.135907,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1.517859
2,0.078811,0.596582,-1.105624,-0.879606,0.808706,0.301522,0,0,1,0,...,1,0,0,0,0,0,0,1,0,2.492872
3,1.029921,-0.202946,-0.360898,-0.879606,-1.566947,-0.857787,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1.346275
4,0.230124,0.596582,-0.733261,-0.879606,-0.254086,-0.029709,0,0,1,0,...,1,0,0,0,0,1,0,0,0,2.009034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,-0.202199,1.396110,-0.733261,-0.879606,1.121292,-1.023402,1,0,1,1,...,1,0,0,1,0,0,0,0,0,1.411111
743,-1.520784,-0.202946,0.383828,0.161703,0.433603,1.792063,1,1,1,0,...,1,0,0,1,0,0,0,0,0,3.372611
744,0.338204,0.996346,-0.733261,-0.879606,0.308569,0.798369,0,0,1,0,...,1,0,0,0,0,0,0,1,0,1.926075
745,1.246082,1.795874,-0.733261,2.244320,0.371086,-0.360940,1,0,1,0,...,1,0,0,0,0,0,1,0,0,3.252402


# Generating client data 

In [15]:
#Setup 1
nC = 3 
samples_per_client= 50

#Setup 2
# nC = 3 
# samples_per_client= 200
# 
# #Setup 3
# nC = 12 
# samples_per_client= 50

# nC = 1 # number of clients
# samples_per_client= 600

data_chunks = np.array_split(main_dataset[0: nC* samples_per_client], nC)  # n chunks for n clients

In [16]:
project_root = '/local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM'
exp_name=f'data/ihdp{nC}_{samples_per_client}'
save_folder = f'{project_root}/{exp_name}'
os.makedirs(save_folder, exist_ok=True)


In [17]:
import pickle

for client_no in range(nC):
    save_loc = f'{save_folder}/client{client_no}_data.pkl'
    print('Saving labels at', save_loc, len(data_chunks[client_no]))
    with open(save_loc, 'wb') as f:
        pickle.dump(data_chunks[client_no], f)
    
    
    save_loc = f'{save_folder}/client{client_no}_data.csv'        
    data_chunks[client_no].to_csv(save_loc, index=False)


Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/ihdp3_50/client0_data.pkl 50
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/ihdp3_50/client1_data.pkl 50
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/ihdp3_50/client2_data.pkl 50


# Creating test dataset

In [18]:
test_chunk= main_dataset[nC* samples_per_client:]
test_chunk

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x18,x19,x20,x21,x22,x23,x24,x25,treatment,y_factual
150,0.684062,0.596582,-0.360898,0.161703,-0.879258,-0.195324,0,0,1,0,...,1,1,0,0,0,0,0,0,0,2.143021
151,-0.059533,-0.602710,-0.360898,0.161703,-1.004292,-0.360940,0,0,1,0,...,1,1,0,0,0,0,0,0,1,9.066118
152,0.511133,-0.602710,-0.360898,-0.879606,-0.379120,0.135907,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1.906684
153,-0.310280,0.596582,-0.733261,-0.879606,-1.629464,-0.195324,1,0,0,0,...,1,0,0,0,0,0,1,0,0,2.044082
154,0.900224,1.396110,-0.360898,2.244320,-2.004568,0.632754,0,0,0,0,...,1,1,0,0,0,0,0,0,0,3.526472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,-0.202199,1.396110,-0.733261,-0.879606,1.121292,-1.023402,1,0,1,1,...,1,0,0,1,0,0,0,0,0,1.411111
743,-1.520784,-0.202946,0.383828,0.161703,0.433603,1.792063,1,1,1,0,...,1,0,0,1,0,0,0,0,0,3.372611
744,0.338204,0.996346,-0.733261,-0.879606,0.308569,0.798369,0,0,1,0,...,1,0,0,0,0,0,0,1,0,1.926075
745,1.246082,1.795874,-0.733261,2.244320,0.371086,-0.360940,1,0,1,0,...,1,0,0,0,0,0,1,0,0,3.252402


In [19]:
test_chunk.columns

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
       'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21',
       'x22', 'x23', 'x24', 'x25', 'treatment', 'y_factual'],
      dtype='object')

In [20]:
save_loc=  f'{save_folder}/test_data.pkl'

print('Saving labels at', save_loc)
with open(save_loc, 'wb') as f:
    pickle.dump(test_chunk, f)
    
save_loc = f'{save_folder}/test_data.csv'        
test_chunk.to_csv(save_loc, index=False)

Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/ihdp3_50/test_data.pkl


Saving labels at ./dummy.pkl
